In [ ]:
import os
from bs4 import BeautifulSoup
from russpelling import *
import pandas as pd
import spacy
from spaczz.matcher import FuzzyMatcher
import enchant
from statistics import mean

dictionary = enchant.Dict("ru_RU")

#Директория с файлами распознанных источников 
os.chdir('C:/Users/YOUR CODE HERE')


list_of_files = os.listdir()


def listmerge3(lstlst):
    all = []
    for lst in lstlst:
        all.extend(lst)
    return all


# очистка текста от лишних символов html-формата

Collection_of_LDA_words_counters_for_text = []
Collection_of_FUZZED_words_counters_for_text = []
Collection_of_recognized_percents = []

for file in list_of_files:
    if file in ["adj-with-ija.txt", "fuzzy_patterns.txt", "ID_patterns_rebuild_1.csv"]:
        continue
    else:
        f = open(file, encoding='utf-8', mode='r')
        html = f.read()
        soup = BeautifulSoup(html, features="html.parser")

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

        text_old = '\n'.join(chunk for chunk in chunks if chunk)

        text_old = text_old.split('\n')

        text_old = [text.split(' ') for text in text_old]

        text_old = listmerge3(text_old)

        text_new_orth = [normalize(word) for word in text_old]

        text_new_orth = [word.strip('.') for word in text_new_orth]
        text_new_orth = [word.strip('_') for word in text_new_orth]
        text_new_orth = [word.strip(',') for word in text_new_orth]
        text_new_orth = [word for word in text_new_orth if word != '']

        # загрузка предобученной модели SpaCy

        nlp = spacy.load("ru_core_news_sm")
        matcher = FuzzyMatcher(nlp.vocab)

        # чтение паттернов Fuzzy-Match, подготовка небольшого датасета

        f = open('fuzzy_patterns.txt', encoding='utf-8', mode='r')
        text_a = f.read()
        data = text_a.split('____________________')
        data = [elem.replace('\n', ' ') for elem in data]
        data = [elem.split(' ') for elem in data]
        data = [set(elem) for elem in data]
        list_of_patterns = []
        for elem in data:
            elem = list(elem)
            elem.pop(0)
            list_of_patterns.append(elem)

        # импорт таблицы с ID для паттернов (необходимо для дальнейшего восстановления слов по ним)

        df = pd.read_csv('ID_patterns_rebuild_1.csv', encoding="cp1251", delimiter=';')

        df['Формы'] = list_of_patterns

        # добавление наших паттернов в шаблоны Fuzzy-Matcher'a

        for i in range(0, len(list_of_patterns)):
            pattern_forms = [nlp(word) for word in list_of_patterns[i]]
            matcher.add(str(i), pattern_forms, kwargs=[{"min_r2": 80}])

        matcher.add('31', [nlp("проц")], kwargs=[{"min_r2": 80}])
        df.loc[len(df.index)] = [31, 'проц', ['проц']]

        # проверка слов на орфографические ошибки, сравнение не прошедших проверку слов с паттернами Fuzzy-match

        word_counter = 0
        words_to_pop_indx = []
        for word in text_new_orth:
            check = dictionary.check(word)
            doc = nlp(word)
            if check == False:
                match_operation = matcher(doc)
                if len(match_operation) == 0:
                    words_to_pop_indx.append(word_counter)
                else:
                    text_new_orth[word_counter] = df.at[int(match_operation[0][0]), "Паттерн"]

            word_counter += 1

        text_new_orth_FUZZ_filter = [text_new_orth[i] for i in range(0, len(text_new_orth)) if i not in words_to_pop_indx]

        final_text = ' '.join(text_new_orth_FUZZ_filter)


        # сохранение файлов в новую директорию


        file_path = "C:/Users/YOUR CODE HERE/" + file[:-4] + '.txt'

        with open(file_path, "w") as f_1:
            f_1.write(final_text)
            f_1.close()

        f.close()
